# Ejercicio de evaluación de Criptosistemas simétricos 

Alumna: Blanca Cano Camarero

## MiniAES

Se muestran a continuación varias celdas de código auxiliar utilizado para el cálculo del algoritmo de MiniAES

In [1]:
F.<xi> = GF(2^4, modulus=GF(2)[x](x^4+x+1))
F.modulus()

x^4 + x + 1

In [2]:
# Definición de las respectivas funciones 

def toNumber(mn):
    n = ''
    for i in [0,2,1,3]:
        n=n+''.join(map(str, mn[i]))
    return Integer(n, base=2), n


In [3]:
def nibblsub(ll):
    xx = vector(ll)
    if xx != 0:
        xx = vector(reversed(vector(F(vector(reversed(xx)))^(-1))))
    return list(xx*matrix(GF(2),4,[0,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1]) + vector(GF(2),[0,0,1,1]))
def key_schedule(kk): 
    ww= list(reversed(vector(GF(2),kk.bits())))
    while len(ww)<16:
        ww = [GF(2)(0)]+ww
    ww = [[ww[0],ww[1],ww[2],ww[3]],
          [ww[4],ww[5],ww[6],ww[7]],
          [ww[8],ww[9],ww[10],ww[11]],
          [ww[12],ww[13],ww[14],ww[15]]] 
    ww.append(list(vector(ww[0]) + vector(nibblsub(ww[3])) + vector(GF(2),[0,0,0,1]))) 
    ww.append(list(vector(ww[1]) + vector(ww[4])))
    ww.append(list(vector(ww[2]) + vector(ww[5])))
    ww.append(list(vector(ww[3]) + vector(ww[6])))                                                         
    ww.append(list(vector(ww[4]) + vector(nibblsub(ww[7])) + vector(GF(2),[0,0,1,0]))) 
    ww.append(list(vector(ww[5]) + vector(ww[8])))
    ww.append(list(vector(ww[6]) + vector(ww[9])))
    ww.append(list(vector(ww[7]) + vector(ww[10])))   
    return ww

def mixcolumn(ll):
    A = matrix(F,2,[[xi+1,xi],[xi,xi+1]])

    b0 = ll[0][0]*xi^3 + ll[0][1]*xi^2 + ll[0][2]*xi + ll[0][3]
    b1 = ll[1][0]*xi^3 + ll[1][1]*xi^2 + ll[1][2]*xi + ll[1][3]
    b2 = ll[2][0]*xi^3 + ll[2][1]*xi^2 + ll[2][2]*xi + ll[2][3]
    b3 = ll[3][0]*xi^3 + ll[3][1]*xi^2 + ll[3][2]*xi + ll[3][3]
    
    
    B = matrix(F,2,[[b0,b2],[b1,b3]])

    return A*B

def shiftrow(ll):
    return [ll[0],ll[3],ll[2],ll[1]]

def addroundkey(ll,kk): 
    return [list(vector(ll[0])+vector(kk[0])),
            list(vector(ll[1])+vector(kk[1])),
            list(vector(ll[2])+vector(kk[2])),
            list(vector(ll[3])+vector(kk[3]))]    

def NumberToMatriz (binario):
    '''Transforma un número binario de la forma 10001001 
    en una matriz 4x4, añade los ceros delante'''
    nl = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    n = list(map(int,reversed(str(binario))))
    n_length = len(n)
    ind = 3
    last = 0
    cnt = 0
    for ind in range(3,-1,-1):
        for i in range(0,4):
            if cnt < n_length:
                nl[ind][3-i] = n[cnt]
                cnt +=1
            else:
                return nl
    return nl



## Ejercicio 

Considerando el cifrado en bloques MiniAES descrito en el ejercicio 2.

### Apartado primero
1. Calcula $E_{dni}(0x01234567)$ usando el modo CBC e IV=$0x0001$.

Donde el mensaje a cifrar es $0x01234567$

In [4]:
mm = 0x01234567 #mensaje 
kk = 75577392 % 65536 # clave 
tam = 16 

# Calulamos k_o
ww =key_schedule(kk)
mensaje = mm.bits()
# añadimos ceros hasta completar un bloque
longitud = 16-len(mensaje)%tam
for i in range(longitud+2, tam):
    mensaje.append(0)

mensaje = list(reversed(mensaje))
mc = 0
ms_length = len(mensaje)
c = 1
sigma
c_ms = '0000000000000001'

for i in range(0, ms_length, tam):
    if(i == ms_length-tam):
        m = mensaje[i:]
    else:
        m = mensaje[i:tam]
    print(f"\n--- E_k(m_{i}:{i+tam}) ---")
    indice_k_sigma = 0
    cnt = 0
    m='0b'+''.join(map(str, m))
    m = format(Integer(m), 'b')
    n = str(c^^Integer(m))
    nl = NumberToMatriz(n)
    ## Vamos concatenandando E_k
    # sigma k0
    sigma = addroundkey(nl,[ww[indice_k_sigma+0],
                            ww[indice_k_sigma+1],
                            ww[indice_k_sigma+2],
                            ww[indice_k_sigma+3]])
    indice_k_sigma += 4
    print(f'sigma_k_{cnt}({nl}) = ', sigma)
    cnt +=1
    # gama 
    gamma = [nibblsub(ele) for ele in sigma]
    print(f'gamma{sigma}) = \n', gamma)
    # pi 
    pi = shiftrow(gamma)
    print(f'pi({gamma}) = \n', pi)
    # tetha miscolumn
    mc = []
    for row in mixcolumn(pi):
        for e in row:
            mc.append(list(vector(e)))  
    print(f'mix column ({pi}) = \n', mc)
    # sigma k1
    sigma = addroundkey(mc,[ww[indice_k_sigma+0],
                            ww[indice_k_sigma+1],
                            ww[indice_k_sigma+2],
                            ww[indice_k_sigma+3]])
    indice_k_sigma += 4
    print(f'sigma_k_{cnt}({mc}) = ', sigma)
    cnt +=1
    # gama 
    gamma = [nibblsub(ele) for ele in sigma]
    print(f'gamma{sigma}) = \n', gamma)
    # pi 
    pi = shiftrow(gamma)
    print(f'pi({gamma}) = \n', pi)
    # sigma k2
    sigma = addroundkey(pi,[ww[indice_k_sigma+0],
                            ww[indice_k_sigma+1],
                            ww[indice_k_sigma+2],
                            ww[indice_k_sigma+3]])
    print(f'sigma_k_{cnt}({pi}) = ', sigma)

    # actualizamos el valor de c
    c,t = toNumber(sigma)
    c_ms = c_ms + t


print(f"\nSe tiene que E_{kk}({mm}) = \n{c_ms}")




--- E_k(m_0:16) ---
sigma_k_0([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0]]) =  [[0, 0, 1, 1], [1, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0]]
gamma[[0, 0, 1, 1], [1, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0]]) = 
 [[0, 1, 1, 1], [1, 1, 1, 0], [1, 0, 0, 0], [1, 1, 1, 1]]
pi([[0, 1, 1, 1], [1, 1, 1, 0], [1, 0, 0, 0], [1, 1, 1, 1]]) = 
 [[0, 1, 1, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 1, 1, 0]]
mix column ([[0, 1, 1, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 1, 1, 0]]) = 
 [[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0]]
sigma_k_1([[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0]]) =  [[0, 0, 1, 1], [1, 0, 1, 1], [1, 0, 0, 1], [1, 1, 1, 0]]
gamma[[0, 0, 1, 1], [1, 0, 1, 1], [1, 0, 0, 1], [1, 1, 1, 0]]) = 
 [[0, 1, 1, 1], [0, 1, 1, 0], [1, 1, 1, 0], [0, 1, 0, 1]]
pi([[0, 1, 1, 1], [0, 1, 1, 0], [1, 1, 1, 0], [0, 1, 0, 1]]) = 
 [[0, 1, 1, 1], [0, 1, 0, 1], [1, 1, 1, 0], [0, 1, 1, 0]]
sigma_k_2([[0, 1, 1, 1], [0, 1, 0, 1], [1, 1, 1, 0], [0, 1, 1, 0]]) =  [[1, 1, 1, 0], [0, 1, 0, 1], 

### Apartado segundo

2. Calcula $E_{dni}(0x01234567)$ usando el modo CFC, r=11, y vector de inicialización IV=$0x0001$.

In [5]:
# Funciones auxiliares 

def msb(tam, number):
    return number[:tam]

def lsb(tam, number):
    return number[-tam:]

In [6]:
mm = 0x01234567 #mensaje 
kk = 75577392 % 65536 # clave 
tam = 11 

# Calulamos k_o
ww =key_schedule(kk)
mensaje = mm.bits()
# añadimos ceros hasta completar un bloque
longitud = 16-len(mensaje)%tam
for i in range(longitud+2, tam):
    mensaje.append(0)

mensaje = list(reversed(mensaje))
mc = 0
ms_length = len(mensaje)
c = 1
sigma
c_ms = '0000000000000001'
x = 1
for i in range(0, ms_length, tam):
    if(i == ms_length-tam):
        m = mensaje[i:]
    else:
        m = mensaje[i:tam]
    print(f"\n--- E_k(m_{i}:{i+tam}) ---")
    indice_k_sigma = 0
    cnt = 0
    nl = NumberToMatriz(x)
    print("nl", nl)
    ## Vamos concatenandando E_k
    # sigma k0
    sigma = addroundkey(nl,[ww[indice_k_sigma+0],
                            ww[indice_k_sigma+1],
                            ww[indice_k_sigma+2],
                            ww[indice_k_sigma+3]])
    indice_k_sigma += 4
    print(f'sigma_k_{cnt}({nl}) = ', sigma)
    cnt +=1
    # gama 
    gamma = [nibblsub(ele) for ele in sigma]
    print(f'gamma{sigma}) = \n', gamma)
    # pi 
    pi = shiftrow(gamma)
    print(f'pi({gamma}) = \n', pi)
    # tetha miscolumn
    mc = []
    for row in mixcolumn(pi):
        for e in row:
            mc.append(list(vector(e)))  
    print(f'mix column ({pi}) = \n', mc)
    # sigma k1
    sigma = addroundkey(mc,[ww[indice_k_sigma+0],
                            ww[indice_k_sigma+1],
                            ww[indice_k_sigma+2],
                            ww[indice_k_sigma+3]])
    indice_k_sigma += 4
    print(f'sigma_k_{cnt}({mc}) = ', sigma)
    cnt +=1
    # gama 
    gamma = [nibblsub(ele) for ele in sigma]
    print(f'gamma{sigma}) = \n', gamma)
    # pi 
    pi = shiftrow(gamma)
    print(f'pi({gamma}) = \n', pi)
    # sigma k2
    sigma = addroundkey(pi,[ww[indice_k_sigma+0],
                            ww[indice_k_sigma+1],
                            ww[indice_k_sigma+2],
                            ww[indice_k_sigma+3]])
    print(f'sigma_k_{cnt}({pi}) = ', sigma)

    # actualizamos el valor de c y x
    c,t = toNumber(sigma)
    mas_significativo = msb(tam, t)
    menos_significativo = lsb(16-tam, t)
    c = str(Integer(Integer(m, base=2) + Integer(mas_significativo, base=2)).binary())
    c_ms = c_ms + c
    x = menos_significativo + c 
    print(f"x_s = {x}") 


print(f"\nSe tiene que E_{kk}({mm}) = \n{c_ms}")


--- E_k(m_0:11) ---
nl [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]]
sigma_k_0([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]]) =  [[0, 0, 1, 1], [1, 0, 0, 0], [0, 0, 1, 1], [0, 0, 0, 1]]
gamma[[0, 0, 1, 1], [1, 0, 0, 0], [0, 0, 1, 1], [0, 0, 0, 1]]) = 
 [[0, 1, 1, 1], [1, 1, 0, 0], [0, 1, 1, 1], [1, 0, 0, 0]]
pi([[0, 1, 1, 1], [1, 1, 0, 0], [0, 1, 1, 1], [1, 0, 0, 0]]) = 
 [[0, 1, 1, 1], [1, 0, 0, 0], [0, 1, 1, 1], [1, 1, 0, 0]]
mix column ([[0, 1, 1, 1], [1, 0, 0, 0], [0, 1, 1, 1], [1, 1, 0, 0]]) = 
 [[0, 1, 0, 1], [0, 1, 0, 0], [1, 0, 1, 0], [1, 0, 0, 1]]
sigma_k_1([[0, 1, 0, 1], [0, 1, 0, 0], [1, 0, 1, 0], [1, 0, 0, 1]]) =  [[0, 1, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0], [0, 0, 1, 1]]
gamma[[0, 1, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0], [0, 0, 1, 1]]) = 
 [[0, 0, 0, 1], [1, 1, 0, 1], [0, 0, 1, 1], [0, 1, 1, 1]]
pi([[0, 0, 0, 1], [1, 1, 0, 1], [0, 0, 1, 1], [0, 1, 1, 1]]) = 
 [[0, 0, 0, 1], [0, 1, 1, 1], [0, 0, 1, 1], [1, 1, 0, 1]]
sigma_k_2([[0, 0, 0, 1], [0, 1, 1, 1], [